# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from math import floor

# Parameters

In [ ]:
stock_symbol = "TCS.NS" #"^NSEI"

base_slow = 26
base_fast = 12
base_smooth = 9

slow_range = range(10, 31, 2)
fast_range = range(5, 21, 2)
smooth_range = range(5, 16, 2)

# Parameter Grid

In [ ]:
param_grid = pd.DataFrame([(slow, fast, smooth) for slow in slow_range for fast in fast_range for smooth in smooth_range], columns=['slow', 'fast', 'smooth'])
param_grid

,slow,fast,smooth
0,10,5,5
1,10,5,7
2,10,5,9
3,10,5,11
4,10,5,13
...,...,...,...
523,30,19,7
524,30,19,9
525,30,19,11
526,30,19,13


# Functions

## Historical Data

In [ ]:
def get_historical_data(symbol):
    df = yf.Ticker(symbol)
    df = df.history(period="max")
    del df["Dividends"]
    del df["Stock Splits"]

    traindf = df[df.index < "2021-01-01"]
    traindf = traindf[traindf.index >= "2015-01-01"]
    validationdf = df[df.index < "2023-01-01"]
    validationdf = validationdf[validationdf.index >= "2021-01-01"]
    testdf = df[df.index >= "2023-01-01"]

    return traindf, validationdf, testdf

## MACD Calculations

In [ ]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()

    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})

    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)

    return df

## Implementing MACD Strategy

In [ ]:
def implement_macd_strategy(prices, data):
    buy_price = []
    sell_price = []
    macd_signal = []
    signal = 0

    for i in range(len(data)):
        if data['macd'][i] > data['signal'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif data['macd'][i] < data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)

    return buy_price, sell_price, macd_signal

## Plot MACD Indicator

In [ ]:
def plot_macd_indicator(stock_data, macd_data):
  ax1 = plt.subplot2grid((11,1), (0,0), rowspan = 5, colspan = 1)
  ax2 = plt.subplot2grid((11,1), (6,0), rowspan = 5, colspan = 1)

  ax1.plot(stock_data['Close'], color = 'skyblue', linewidth = 2, label = f'{stock_symbol}')
  ax1.plot(stock_data.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
  ax1.plot(stock_data.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
  ax1.legend()
  ax1.set_title('MACD SIGNALS')
  ax2.plot(macd_data['macd'], color = 'grey', linewidth = 1.5, label = 'MACD')
  ax2.plot(macd_data['signal'], color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

  for i in range(len(macd_data)):
      if str(macd_data['hist'][i])[0] == '-':
          ax2.bar(macd_data.index[i], macd_data['hist'][i], color = '#ef5350')
      else:
          ax2.bar(macd_data.index[i], macd_data['hist'][i], color = '#26a69a')

  plt.legend(loc = 'lower right')
  plt.show()

## Creating Positions in the Market

In [ ]:
def creating_position(macd_signal, train_data, macd_data):
  position = []
  for i in range(len(macd_signal)):
      if macd_signal[i] > 1:
          position.append(0)
      else:
          position.append(1)

  for i in range(len(train_data['Close'])):
      if macd_signal[i] == 1:
          position[i] = 1
      elif macd_signal[i] == -1:
          position[i] = 0
      else:
          position[i] = position[i-1]

  macd = macd_data['macd']
  signal = macd_data['signal']
  close_price = train_data['Close']
  macd_signal = pd.DataFrame(macd_signal).rename(columns = {0:'macd_signal'}).set_index(train_data.index)
  position = pd.DataFrame(position).rename(columns = {0:'macd_position'}).set_index(train_data.index)

  frames = [close_price, macd, signal, macd_signal, position]
  strategy = pd.concat(frames, join = 'inner', axis = 1)

  return strategy

## Backtesting

In [ ]:
def backtesting(strategy, train_data):
  stock_ret = pd.DataFrame(np.diff(train_data['Close'])).rename(columns = {0:'returns'})
  macd_strategy_ret = []

  for i in range(len(stock_ret)):
      try:
          returns = stock_ret['returns'][i]*strategy['macd_position'][i]
          macd_strategy_ret.append(returns)
      except:
          pass

  macd_strategy_ret_df = pd.DataFrame(macd_strategy_ret).rename(columns = {0:'macd_returns'})

  investment_value = 100000
  number_of_stocks = floor(investment_value/train_data['Close'][0])
  macd_investment_ret = []

  for i in range(len(macd_strategy_ret_df['macd_returns'])):
      returns = number_of_stocks*macd_strategy_ret_df['macd_returns'][i]
      macd_investment_ret.append(returns)

  macd_investment_ret_df = pd.DataFrame(macd_investment_ret).rename(columns = {0:'investment_returns'})
  total_investment_ret = round(sum(macd_investment_ret_df['investment_returns']), 2)
  profit_percentage = floor((total_investment_ret/investment_value)*100)

  return total_investment_ret, profit_percentage

## Best MACD Parameters

In [ ]:
def find_best_macd_parameters(param_grid, traindf):
  best_params = None
  max_profit = -float('inf')
  for index, row in param_grid.iterrows():
    slow, fast, smooth = row['slow'], row['fast'], row['smooth']
    macddf = get_macd(traindf['Close'], slow, fast, smooth)
    print(f"Parameters: slow={slow}, fast={fast}, smooth={smooth}")
    print("MACD Parameters:")
    print(macddf)
    buy_price, sell_price, macd_signal = implement_macd_strategy(traindf['Close'], macddf)
    print("Buy Prices:")
    print(buy_price)
    print("Sell Prices:")
    print(sell_price)
    print("MACD Signal:")
    print(macd_signal)
    # plot_macd_indicator(traindf, macddf)
    strategy = creating_position(macd_signal, traindf, macddf)
    print("Strategy:")
    print(strategy)
    tot_inv_ret, profit_per = backtesting(strategy, traindf)
    print(f'Profit gained from the MACD strategy by investing Rs. 1 Lakh : {tot_inv_ret}')
    print(f'Profit percentage of the MACD strategy : {profit_per}%')

    if profit_per > max_profit:
            max_profit = profit_per
            best_params = (slow, fast, smooth)

  return best_params, max_profit

# Data Visualization

## Stock Data Visualization

In [ ]:
traindf, validationdf, testdf = get_historical_data(stock_symbol)

In [ ]:
traindf

,Open,High,Low,Close,Volume
Date,,,,,
2015-01-01 00:00:00+05:30,1066.400631,1066.400631,1055.599534,1057.489746,366830
2015-01-02 00:00:00+05:30,1059.753880,1076.350163,1059.587750,1071.572754,925740
2015-01-05 00:00:00+05:30,1072.216855,1080.068383,1048.807508,1055.288208,1754242
2015-01-06 00:00:00+05:30,1050.655824,1050.655824,1013.641261,1016.383118,2423784
2015-01-07 00:00:00+05:30,1026.104472,1029.905588,1000.119499,1004.377625,2636332
...,...,...,...,...,...
2020-12-24 00:00:00+05:30,2734.568844,2746.178398,2710.315833,2734.897949,1807144
2020-12-28 00:00:00+05:30,2735.508971,2772.828412,2727.988671,2753.745605,2108994
2020-12-29 00:00:00+05:30,2744.908905,2774.990099,2744.908905,2754.779297,1994151


In [ ]:
validationdf

,Open,High,Low,Close,Volume
Date,,,,,
2021-01-01 00:00:00+05:30,2707.307986,2763.710235,2706.367948,2752.664795,2681440
2021-01-04 00:00:00+05:30,2773.110000,2867.818756,2764.602616,2857.196289,5113293
2021-01-05 00:00:00+05:30,2857.337980,2927.511685,2857.337980,2907.535889,5801309
2021-01-06 00:00:00+05:30,2914.115736,2926.806241,2855.081346,2868.523926,3726716
2021-01-07 00:00:00+05:30,2890.614595,2896.113905,2820.346809,2850.945068,3717827
...,...,...,...,...,...
2022-12-26 00:00:00+05:30,3110.250378,3152.303476,3107.071051,3133.902100,870157
2022-12-27 00:00:00+05:30,3149.605604,3154.037420,3113.284798,3140.260498,835883
2022-12-28 00:00:00+05:30,3130.915732,3146.619492,3107.986338,3137.948730,910795


In [ ]:
testdf

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-02 00:00:00+05:30,3141.705913,3147.342002,3116.849683,3142.139404,709547
2023-01-03 00:00:00+05:30,3133.035162,3198.547582,3126.580309,3190.214111,1245178
2023-01-04 00:00:00+05:30,3185.733941,3205.628662,3165.983874,3193.393066,1231668
2023-01-05 00:00:00+05:30,3208.374286,3215.214647,3161.889371,3189.973145,1826057
2023-01-06 00:00:00+05:30,3179.278961,3180.772305,3082.937174,3094.064697,2488376
...,...,...,...,...,...
2024-03-06 00:00:00+05:30,3994.050049,4072.000000,3958.449951,4064.300049,1744204
2024-03-07 00:00:00+05:30,4065.000000,4124.000000,4037.699951,4108.600098,2688905
2024-03-11 00:00:00+05:30,4089.000000,4153.000000,4089.000000,4122.350098,2304068


## MACD Parameter Grid Visualization

In [ ]:
for i in param_grid.iterrows():
  print(i, "\n")

(0, slow      10
fast       5
smooth     5
Name: 0, dtype: int64) 

(1, slow      10
fast       5
smooth     7
Name: 1, dtype: int64) 

(2, slow      10
fast       5
smooth     9
Name: 2, dtype: int64) 

(3, slow      10
fast       5
smooth    11
Name: 3, dtype: int64) 

(4, slow      10
fast       5
smooth    13
Name: 4, dtype: int64) 

(5, slow      10
fast       5
smooth    15
Name: 5, dtype: int64) 

(6, slow      10
fast       7
smooth     5
Name: 6, dtype: int64) 

(7, slow      10
fast       7
smooth     7
Name: 7, dtype: int64) 

(8, slow      10
fast       7
smooth     9
Name: 8, dtype: int64) 

(9, slow      10
fast       7
smooth    11
Name: 9, dtype: int64) 

(10, slow      10
fast       7
smooth    13
Name: 10, dtype: int64) 

(11, slow      10
fast       7
smooth    15
Name: 11, dtype: int64) 

(12, slow      10
fast       9
smooth     5
Name: 12, dtype: int64) 

(13, slow      10
fast       9
smooth     7
Name: 13, dtype: int64) 

(14, slow      10
fast       9
smooth   

# Driver Code

In [ ]:
best_params, max_profit = find_best_macd_parameters(param_grid, traindf)
print(f"Best MACD Parameters: slow={best_params[0]}, fast={best_params[1]}, smooth={best_params[2]}")
print(f"Highest Profit Percentage: {max_profit}%")

Streaming output truncated to the last 5000 lines.
2015-01-06 00:00:00+05:30   -9.970667  -0.710288  -9.260379
2015-01-07 00:00:00+05:30  -20.123979  -3.136999 -16.986980
...                               ...        ...        ...
2020-12-24 00:00:00+05:30   94.040497  65.879082  28.161414
2020-12-28 00:00:00+05:30  100.304135  70.182214  30.121921
2020-12-29 00:00:00+05:30  102.159343  74.179355  27.979988
2020-12-30 00:00:00+05:30   95.693851  76.868667  18.825184
2020-12-31 00:00:00+05:30   77.912578  76.999156   0.913422

[1480 rows x 3 columns]
Buy Prices:
[nan, 1071.57275390625, nan, nan, nan, nan, nan, nan, nan, 1047.685546875, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1075.9442138671875, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1099.504150390625, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan